In [ ]:
import pandas as pd
import numpy
import matplotlib.pyplot as plt
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [ ]:
dataset = pd.read_csv('regression_dataset.csv')
dataset.head()

,cases,latitude,longitude,maxtempC,mintempC,totalSnow_cm,sunHour,uvIndex,moon_illumination,DewPointC,FeelsLikeC,HeatIndexC,WindChillC,WindGustKmph,cloudcover,humidity,pressure,tempC,visibility,winddirDegree,windspeedKmph,year,month,density_per_km,precipMM
0,2,-34.603684,-58.381559,24,7,0.0,11.6,5,87,6,14,15,14,16,0,59,1018,24,10,189,10,2016,3,0.505636,0.0
1,0,-34.603684,-58.381559,24,7,0.0,11.6,5,87,6,14,15,14,16,0,59,1018,24,10,189,10,2016,3,0.505636,0.0
2,1,-34.603684,-58.381559,24,7,0.0,11.6,5,87,6,14,15,14,16,0,59,1018,24,10,189,10,2016,3,0.505636,0.0
3,0,-34.603684,-58.381559,22,10,0.0,11.5,5,76,10,14,16,14,27,34,72,1025,22,10,99,18,2016,4,0.505636,0.0
4,1,-34.603684,-58.381559,22,10,0.0,11.5,5,76,10,14,16,14,27,34,72,1025,22,10,99,18,2016,4,0.505636,0.0


In [ ]:
# split into train and test sets
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train = dataset.iloc[0:train_size,:]
test = dataset.iloc[train_size:len(dataset),:]
print(len(train), len(test))

69040 34005


In [ ]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset.iloc[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset.iloc[i + look_back, 0])
	return numpy.array(dataX), numpy.array(dataY)

In [ ]:
# reshape into X=t and Y=t+1
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [ ]:
# reshape input to be [samples, time steps, features]
trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [ ]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)

Epoch 1/100
69038/69038 - 76s - loss: 519342.5312
Epoch 2/100
69038/69038 - 76s - loss: 518093.4062
Epoch 3/100
69038/69038 - 75s - loss: 517701.6875
Epoch 4/100
69038/69038 - 75s - loss: 517509.6562
Epoch 5/100
69038/69038 - 75s - loss: 517391.8750
Epoch 6/100
69038/69038 - 75s - loss: 517249.5000
Epoch 7/100
69038/69038 - 76s - loss: 517283.9062
Epoch 8/100
69038/69038 - 75s - loss: 517313.6250
Epoch 9/100
69038/69038 - 76s - loss: 517258.0312
Epoch 10/100
69038/69038 - 76s - loss: 517183.2188
Epoch 11/100
69038/69038 - 76s - loss: 517207.1250
Epoch 12/100
69038/69038 - 77s - loss: 517292.2500
Epoch 13/100
69038/69038 - 75s - loss: 517254.5312
Epoch 14/100
69038/69038 - 75s - loss: 517242.9688
Epoch 15/100
69038/69038 - 76s - loss: 517248.4062
Epoch 16/100
69038/69038 - 76s - loss: 517298.3125
Epoch 17/100
69038/69038 - 77s - loss: 517261.8125
Epoch 18/100
69038/69038 - 78s - loss: 517248.3125
Epoch 19/100
69038/69038 - 75s - loss: 517363.5625
Epoch 20/100
69038/69038 - 74s - loss: 5

In [ ]:
from sklearn.metrics import mean_squared_error
import numpy as np

# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)
# # invert predictions
# trainPredict = scaler.inverse_transform(trainPredict)
# trainY = scaler.inverse_transform([trainY])
# testPredict = scaler.inverse_transform(testPredict)
# testY = scaler.inverse_transform([testY])
# calculate root mean squared error
trainScore = mean_squared_error(trainY, trainPredict)
print('Train Score: %.2f MSE' % (trainScore))
testScore = mean_squared_error(testY, testPredict)
print('Test Score: %.2f MSE' % (testScore))

Train Score: 517015.64 MSE
Test Score: 555395.43 MSE
